In [1]:
import pandas as pd
from pulp import *
import re

#Loading fantasy list into dataframe
players = pd.read_csv(r"C:\Users\dymar\OneDrive\Desktop\DKWeek11.csv", usecols= ['ID', 'Position', 'AvgPointsPerGame', 'Salary', 'Name', 'KeyPlayer'])

#Grouping players to start manipulating the data
availables = players.groupby(["Position", "ID", "AvgPointsPerGame", "Salary", "KeyPlayer"]).agg('count')
availables = availables.reset_index()

#Defining empty salary and point dictionaries to store data for players
salaries = {}
points = {}
keys = {}
lineups_dict = {}

#Creating dataframes to pull out unique values for every position
for pos in availables.Position.unique():
    available_pos = availables[availables.Position == pos]
    salary = list(available_pos[['ID', 'Salary']].set_index("ID").to_dict().values())[0]
    point = list(available_pos[['ID', 'AvgPointsPerGame']].set_index("ID").to_dict().values())[0]
    key = list(available_pos[['ID', 'KeyPlayer']].set_index("ID").to_dict().values())[0]
    
    salaries[pos] = salary
    points[pos] = point
    keys[pos] = key
    
#Defining positional/salary constraints
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 4,
    "TE": 1,
    "DST": 1
                    }

salary_cap = 50000


#How many lineups we want to create and iteration based on constraints
for lineup in range(1,16):
    _vars = {k: LpVariable.dict(k, v, cat='Binary') for k, v in points.items()}

    prob = LpProblem("Fantasy", LpMaximize)
    rewards = []
    costs = []
    key_players = []

    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        key_players += lpSum([keys[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]

#The first lineup is max fantasy points. After that subtract 1.0 from score and set that as new constraint to get multiple lineups.
    prob += lpSum(rewards)
    prob += lpSum(costs) <= salary_cap
    prob += lpSum(key_players) >=0
    if not lineup == 1:
        prob += (lpSum(rewards) <= total_score-1.0)
    prob.solve()

#Setting up and cleaning up data so it is easier to read    
    score= str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    lineupList = []
    
#Making sure we only pull in players that are used in the lineup
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        if v.varValue !=0:
            lineupList.append(v.name)    
    total_score = eval(score)
    lineupList.append(total_score)
    print(lineup, total_score)
    lineups_dict.update({lineup: lineupList})

1 155.5
2 154.48
3 153.48000000000002
4 152.48
5 151.48000000000002
6 150.48
7 149.48
8 148.48
9 147.48
10 146.48000000000002
11 145.48
12 144.48000000000002
13 143.48000000000002
14 142.48
15 141.48


In [2]:
# Creating a dataframe of lineups
df = pd.DataFrame(lineups_dict)

#Transpose data for easy reading
df = df.T

#Creating new columns heads to replace the numbers
newcols = ['DST', 'QB', 'RB1', 'RB2', 'TE', 'WR1', 'WR2', 'WR3', 'WR4', 'Total_Score']
df.columns = newcols

#Create new dataframe incase of an error
df2 = df

#Laying out data for upload to DraftKings. Removing unecessary formatting
positions = ['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE', 'WR4', 'DST']
removeKeys = ['QB_', 'RB_', 'WR_', 'TE_', 'DST_']
for pos in positions:
    for removeKey in removeKeys:
        df2[pos] = df2[pos].str.replace(removeKey, "")
        
#Create new dataframe after formatting for uploading
df3 = df2 [['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE', 'WR4', 'DST', 'Total_Score']]

#Reading ID as a integer instead of string because Fanduel has "_" in front of their file that field is a string.
players['ID'] = players['ID'].astype('str')

#Bringing in player name
playerNameDict = dict(zip(players["ID"],players["Name"]))

#Attaching to last data frame
dfPlayerName = df3

#Iteration
for pos in positions:
    dfPlayerName[pos] = dfPlayerName[pos].replace(playerNameDict)
    
#Check and final results of lineups by name
dfPlayerName



,QB,RB1,RB2,WR1,WR2,WR3,TE,WR4,DST,Total_Score
1,Justin Herbert,Jonathan Taylor,Eli Mitchell,Deebo Samuel,Antonio Brown,Sterling Shepard,Rob Gronkowski,DJ Chark Jr.,Bills,155.5
2,Justin Herbert,Aaron Jones,Darrell Henderson Jr.,Cooper Kupp,Mike Williams,Antonio Brown,Rob Gronkowski,DJ Chark Jr.,Bills,154.48
3,Justin Herbert,Jonathan Taylor,Cordarrelle Patterson,Diontae Johnson,Antonio Brown,Corey Davis,Rob Gronkowski,DJ Chark Jr.,Bills,153.48
4,Josh Allen,Jonathan Taylor,Cordarrelle Patterson,Michael Pittman Jr.,Antonio Brown,Sterling Shepard,Rob Gronkowski,DJ Chark Jr.,Bills,152.48
5,Justin Herbert,Austin Ekeler,Alvin Kamara,Mike Williams,Antonio Brown,Corey Davis,Rob Gronkowski,Deonte Harris,Bills,151.48
6,Justin Herbert,Austin Ekeler,Eli Mitchell,Deebo Samuel,Antonio Brown,Sterling Shepard,Rob Gronkowski,DJ Chark Jr.,Colts,150.48
7,Justin Herbert,Jonathan Taylor,Cordarrelle Patterson,Tyler Lockett,Mike Williams,Antonio Brown,Rob Gronkowski,DJ Chark Jr.,Giants,149.48
8,Justin Herbert,Jonathan Taylor,Antonio Gibson,Cooper Kupp,Antonio Brown,DJ Chark Jr.,Rob Gronkowski,Anthony Miller,Panthers,148.48
9,Justin Herbert,Darrell Henderson Jr.,Eli Mitchell,Cooper Kupp,Michael Pittman Jr.,Antonio Brown,Rob Gronkowski,Anthony Miller,Colts,147.48
10,Justin Herbert,Eli Mitchell,Nyheim Hines,Cooper Kupp,Tyler Lockett,Michael Pittman Jr.,Rob Gronkowski,Antonio Brown,Broncos,146.48


In [3]:
#Create new dataframe after formatting for uploading to Draftkings
df3 = df2 [['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'TE', 'WR4', 'DST', 'Total_Score']]

In [4]:
df3

,QB,RB1,RB2,WR1,WR2,WR3,TE,WR4,DST,Total_Score
1,20289175,20289258,20289302,20289596,20289640,20289666,20290110,20289908,20290392,155.5
2,20289175,20289288,20289292,20289590,20289632,20289640,20290110,20289908,20290392,154.48
3,20289175,20289258,20289278,20289616,20289640,20289682,20290110,20289908,20290392,153.48
4,20289170,20289258,20289278,20289634,20289640,20289666,20290110,20289908,20290392,152.48
5,20289175,20289262,20289270,20289632,20289640,20289682,20290110,20289732,20290392,151.48
6,20289175,20289262,20289302,20289596,20289640,20289666,20290110,20289908,20290393,150.48
7,20289175,20289258,20289278,20289622,20289632,20289640,20290110,20289908,20290407,149.48
8,20289175,20289258,20289294,20289590,20289640,20289908,20290110,20289932,20290391,148.48
9,20289175,20289292,20289302,20289590,20289634,20289640,20290110,20289932,20290393,147.48
10,20289175,20289302,20289550,20289590,20289622,20289634,20290110,20289640,20290402,146.48


In [5]:
# create excel writer object
writer = pd.ExcelWriter('Lineups.xlsx')

# write dataframe to excel
df3.to_excel(writer)

# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
